In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Sequential, Linear, ReLU, MSELoss, L1Loss, CrossEntropyLoss, NLLLoss, LogSoftmax, Softmax
from torch.optim import Adam, RMSprop, SGD
from torch import from_numpy, no_grad, argmax
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
from sklearn import preprocessing

In [20]:
class NNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        ##############
        ##############

        self.fc1 = nn.Linear(25, 6)
        self.fcout = nn.Linear(6, 3)

        ##############
        ##############

    def forward(self, x):
        ##############
        ##############

        x = F.relu(self.fc1(x))
        x = F.softmax(self.fcout(x), dim=1)

        ##############
        ##############
        return x


def main_gpu():
    torch.manual_seed(123)
    BATCH_SIZE = 30
    EPOCHS = 50
    LEARNING_RATE = 0.001

    data = np.load("biofilm_data.npy")
    X = data[:, :-1]
    y = data[:, -1]
    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(X)

    X_train, X_val = torch.from_numpy(X[:int(X.shape[0] * 0.8), :].astype(np.float)).float(), torch.from_numpy(
        X[int(X.shape[0] * 0.8):, :].astype(np.float)).float()
    y_train, y_val = torch.from_numpy(y[:int(X.shape[0] * 0.8)].astype(np.float)).float(), torch.from_numpy(
        y[int(X.shape[0] * 0.8):].astype(np.float)).float()

    dl_training = None
    dl_validation = None

    ##############
    ##############

    dl_training = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
    dl_validation = DataLoader(TensorDataset(X_val, y_val), batch_size=BATCH_SIZE, shuffle=True)

    ##############
    ##############
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    standard_model = NNClassifier().to(device)

    optimizer = Adam(standard_model.parameters(), lr=LEARNING_RATE)  # can be replaced with RMSprop, SGD..
    
    loss_fun = CrossEntropyLoss()

    for epoch in range(EPOCHS):
        epoch_loss = 0.0
        for inputs, true_output in dl_training:
            inputs = inputs.to(device)
            true_output = true_output.type(torch.LongTensor)
            true_output = true_output.to(device)
            
            predictions = None
            loss = None
            ##############
            ##############
            
            predictions = standard_model(inputs)
            loss = loss_fun(predictions, true_output)

            ##############
            ##############
            epoch_loss += float(loss.detach())

            ##############
            ##############

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            ##############
            ##############
        with no_grad():
            standard_model.eval()
            acc = 0.0
            for inputs, true_output in dl_training:
                predictions = standard_model(inputs)

                acc += (argmax(predictions, -1) == true_output).sum()

        with no_grad():
            standard_model.eval()
            val_acc = 0.0
            for inputs, true_output in dl_validation:
                predictions = standard_model(inputs)
                val_acc += (argmax(predictions, -1) == true_output).sum()
        standard_model.train()

        print(["Epoch: %d, Training accuracy: %3.4f, Validation accuracy: %3.4f" % (
            epoch + 1, acc * 100 / len(dl_training.dataset), val_acc * 100 / len(dl_validation.dataset))])

    torch.save(standard_model.state_dict(), "my_classification_model_gpu.pt")  # DO NOT MODIFY THIS LINE!
    ##############
    ##############

    # TODO: add saved model to repository! (no coding necessary, just wanted to get your attention here)

    ##############
    ##############

In [21]:
import timeit

start = timeit.default_timer()

#Your statements here
main_gpu()

stop = timeit.default_timer()

print('Time: ', stop - start) 

cuda:0


AttributeError: 'NNClassifier' object has no attribute 'is_cuda'